# Práctica 1: Reglas de asociación

Imports necesarios para la realización de la práctica

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import mlxtend as mlx
import apyori as apy
import matplotlib as mpl

Pregunta Nº1: Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto. (1 punto)


Pregunta Nº2: ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos (1 punto)

Pregunta Nº3: ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto) consideradas? ¿Por qué? (1 punto)

Pregunta Nº4: Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y min_threshold. Justifica los resultados. (1 punto)


Pregunta Nº5: Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos (1 punto)